In [1]:
from google.colab import drive
drive.mount('/content/MyDrive/')

Drive already mounted at /content/MyDrive/; to attempt to forcibly remount, call drive.mount("/content/MyDrive/", force_remount=True).


In [2]:
# Select TensorFlow 2.0
%tensorflow_version 2.x

In [3]:
# Install Weights and Biases
!pip install wandb

In [4]:
# Imports
from tensorflow.keras.callbacks import TensorBoard

import wandb
from wandb.keras import WandbCallback

In [5]:
# Login to your W&B account
!wandb login

wandb: Currently logged in as: taariqdb (use `wandb login --relogin` to force relogin)


In [6]:
# Initialize your W&B project allowing it to sync with TensorBoard
wandb.init(project="tensorboard-integration-partII", sync_tensorboard=True)
config = wandb.config

wandb: Currently logged in as: taariqdb (use `wandb login --relogin` to force relogin)


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
df = pd.read_csv("/content/MyDrive/MyDrive/Allfalltypesnew.csv")

In [9]:
df = pd.get_dummies(data = df, columns=['DescriptionID'])
df = pd.get_dummies(data = df, columns=['FallTypes'])
df.columns

Index(['AccX', 'AccY', 'AccZ', 'GyrX', 'GyrY', 'GyrZ', 'EulerX', 'EulerY',
       'EulerZ', 'TrialID', 'TaskID', 'Subject', 'Description', 'FTLabels',
       'DescriptionID_1', 'DescriptionID_2', 'DescriptionID_3',
       'DescriptionID_4', 'DescriptionID_5', 'DescriptionID_6',
       'DescriptionID_7', 'DescriptionID_8', 'DescriptionID_9',
       'DescriptionID_10', 'DescriptionID_11', 'DescriptionID_12',
       'DescriptionID_13', 'DescriptionID_14', 'DescriptionID_15',
       'FallTypes_Backward', 'FallTypes_Dampen', 'FallTypes_Forward',
       'FallTypes_Lateral'],
      dtype='object')

In [10]:
from sklearn.preprocessing import StandardScaler
X = df[['AccX', 'AccY', 'AccZ', 'GyrX', 'GyrY', 'GyrZ', 'EulerX', 'EulerY','EulerZ',
       'DescriptionID_1', 'DescriptionID_2', 'DescriptionID_3',
       'DescriptionID_4', 'DescriptionID_5', 'DescriptionID_6',
       'DescriptionID_7', 'DescriptionID_8', 'DescriptionID_9',
       'DescriptionID_10', 'DescriptionID_11', 'DescriptionID_12',
       'DescriptionID_13', 'DescriptionID_14', 'DescriptionID_15',]]
scaler = StandardScaler()
scaled_data = scaler.fit_transform(X)      
X = np.asarray(X)

In [11]:
y = df[['FallTypes_Backward', 'FallTypes_Dampen', 'FallTypes_Forward','FallTypes_Lateral']]
y = np.asarray(y)

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization

In [14]:
from sklearn.metrics import confusion_matrix
from datetime import datetime
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import itertools
import io

In [15]:
labels = ['Backward','Dampen','Forward','Lateral']

In [18]:
from tensorflow.keras.initializers import RandomNormal, Constant# Model with default batch normalization
model = Sequential([
    Dense(64, input_shape=(24,), activation="relu"),
    BatchNormalization(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dense(64, activation='relu'),
    BatchNormalization(
        momentum=0.95, 
        epsilon=0.005,
        beta_initializer=RandomNormal(mean=0.0, stddev=0.05), 
        gamma_initializer=Constant(value=0.9)
    ),
    Dense(4, activation='softmax')
]);
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train,y_train,epochs=200,validation_split=0.25,batch_size=40,verbose=2,callbacks=[WandbCallback(),TensorBoard(log_dir=wandb.run.dir)])

Epoch 1/200


wandb: WARNING When using several event log directories, please call wandb.tensorboard.patch(root_logdir="...") before wandb.init


2605/2605 - 10s - loss: 0.2513 - accuracy: 0.9060 - val_loss: 0.0357 - val_accuracy: 0.9883 - _timestamp: 1648704903.0000 - _runtime: 73.0000 - 10s/epoch - 4ms/step
Epoch 2/200
2605/2605 - 8s - loss: 0.0259 - accuracy: 0.9918 - val_loss: 0.0141 - val_accuracy: 0.9973 - _timestamp: 1648704911.0000 - _runtime: 81.0000 - 8s/epoch - 3ms/step
Epoch 3/200
2605/2605 - 8s - loss: 0.0130 - accuracy: 0.9963 - val_loss: 0.0012 - val_accuracy: 0.9997 - _timestamp: 1648704919.0000 - _runtime: 89.0000 - 8s/epoch - 3ms/step
Epoch 4/200
2605/2605 - 8s - loss: 0.0078 - accuracy: 0.9979 - val_loss: 0.0750 - val_accuracy: 0.9773 - _timestamp: 1648704927.0000 - _runtime: 97.0000 - 8s/epoch - 3ms/step
Epoch 5/200
2605/2605 - 8s - loss: 0.0081 - accuracy: 0.9979 - val_loss: 0.0011 - val_accuracy: 0.9998 - _timestamp: 1648704935.0000 - _runtime: 105.0000 - 8s/epoch - 3ms/step
Epoch 6/200
2605/2605 - 8s - loss: 0.0061 - accuracy: 0.9982 - val_loss: 0.0041 - val_accuracy: 0.9988 - _timestamp: 1648704942.0000 -

In [21]:
#using lstm model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
pred=model.predict(X_test)
accuracy=model.evaluate(X_test,y_test)
print("accuracy is {}".format(accuracy[1]))
wandb.sklearn.plot_confusion_matrix(y_test.argmax(axis=1), pred.argmax(axis=1),labels)
wandb.termlog('Logged confusion matrix.')
wandb.sklearn.plot_class_proportions(y_train.argmax(axis=1), y_test.argmax(axis=1),['Forward','Lateral','Backward','Dampen'])
wandb.termlog('Logged class proportions.')

1086/1086 [==============================] - 2s 2ms/step - loss: 0.0134 - accuracy: 0.9976
accuracy is 0.9975525736808777


wandb: Logged confusion matrix.
wandb: Logged class proportions.
